# Project Milestone 2

Here we will describe the whole pipeline to get all the results we would like to include in the final story (on the final website). We will go through all the different steps and describe as detailed as possible the operations needed.

## **[Preprocessing steps](#Preprocessing)**

As usual the first step consist in several substeps that aims at cleaning and transforming the data. By clicking on the task link, you can access the respective pipeline.
- *[Data exploration and Sanity check](#Sanity_check)* : Explore the dataset, check its consistency and get familiar with the different features/information provided into.
    - Collaborators assigned to that task: ALL.
- *[Data extraction](#extraction)* : Extract the datas of interest that will be further used to perform the tasks related to each idea.
    - Collaborators assigned to that task: Arnaud.
- *[Data augmentation](#augmentation)* : Perform a data augmentation to get more features about the quotations such as the quote field, the nationality of the speaker and so on... These new features will be further used to perform the tasks related to each idea.
    - Collaborators assigned to that task: Jean & Gaelle. 
- *[Quotations and speakers clustering](#clustering)* : Cluster the quotations and the speakers according to the a quotation vector and the added features (data augmentation).
    - Collaborators assigned to that task: Rafaele (RESTART FROM SCRATCH, PLEASE DO SOMETHING INTELLIGIBLE) + Look at website.

## **[Generate the results for the final story](#Results)**

*[Analysis of the way Brexit is perceived](#Brexit)* : The first idea consists in visualizing the the way Brexit is perceived accross different fields as well as accross the different countries in Europe.**JEAN COULD YOU COMPLETE WITH YOUR INITIAL DESCRIPTION ? (PUT SOMETHING APPEALING)**
- Country map
- Bubbles
- Clustering evolution
- Maybe recommandation system
- Evolution of the number of quotations about Brexit


# Before diving into the code 

Make sure you have a **Data** Folder containing all the quotebank datasets.

## Import useful librairies

In [ ]:
import bz2 
import json
import pandas as pd
#from pybrainyquote import *

<a id='Preprocessing'></a>

# Preprocessing steps

<a id='Sanity_check'></a>

## Data exploration and Sanity check

Define which exploration and sanity checks we need to perform:

- Distribution of the number of occurences
- Remove duplicate quotations
- ...

In [ ]:
# RETRIEVE DATA FROM DIFFERENT SOURCES

"""
    Initialize quotes dictionary
    key = author
    values = [quotations]
"""
quotes_dict = {}

"""
    Authors sparse informations that can be already retrieved
    key = author
    values = [info_as_string]
"""
authors_info = {} 

# SOURCE: Brainyquote.com website
brainyquotes = Quote.find_all('brexit')

if brainyquotes is not None:
    for q in branyquotes:
        quotes_dict[q.author].append(q.content)
        authors_info[q.author] = q.info
else:
    print("Error: retrieving data from Brainyquotes")

In [ ]:
# PERFORM THE DIFFERENT SANITY CHECKS

<a id='extraction'></a>

## Data extraction

As mentionned previously, we are planning to analyze the influence of Brexit on different branch as well as analyzing the evolution of feelings towards China. To be able to perform such tasks, we need first to extract the quotations that are talking from Brexit and the ones that are talking about China. To do so we will follow the following pipeline:

1. Both for Brexit and China, define a neighborhood containing all the words that are respectively closely related to Brexit and China. This neighborhood will be a list of words or expressions that are commonly used to refer to Brexit or China. For instance, for China one could actually add to the vocabulary neighborhood the *"the Middle Kingdom"* expression that is often used to refer to China.
2. Both for Brexit and China, select all the quotations for which, at least, one word/expression from the vocabulary neighborhood appears in it.
3. Store the new two datasets in the following files: 
    - `Brexit_quotes.json.bz2`


In [ ]:
# DEFINE BREXIT VOCAB

Brexit_vocab = ["Brexit","brexit"]

"Brexit" in "Brexit is beautiful"

In [ ]:
# DO THE EXTRACTION HERE

# PATHS_TO_FILE = [('Data/quotes-20%d.json.bz2' % i) for i in range(15,21)]
PATHS_TO_FILE = ['Data/quotes-2020.json.bz2']
PATH_TO_OUT = 'Brexit_datas/Brexit_quotes.json.bz2'

with bz2.open(PATH_TO_OUT, 'wb') as d_file:
    for quotebank_data in PATHS_TO_FILE:
        with bz2.open(quotebank_data, 'rb') as s_file:
            for instance in s_file:
                instance = json.loads(instance) # loading a sample
                quotation = instance['quotation'] # extracting the quotation
                if any(substring in quotation for substring in Brexit_vocab):
                    d_file.write((json.dumps(instance)+'\n').encode('utf-8')) # writing in the new file

In [ ]:
quotebank_brexit_test = pd.read_json('Brexit_datas/Brexit_quotes.json.bz2',compression="bz2",lines=True)
quotebank_brexit_test.head()

<a id='augmentation'></a>

## Data augmentation

When we will generate the results for the final story, we will need more information than the initial features we have. The further analysis will require to have access to other features such as the topic of the quotation, the sentiment that carries the quotation, some information about the author and so on. The main idea is to add new features to the existing dataset or only to the data of interest. To do so, we will follow the following pipeline for each quotation:

1. **Add features related to the author** : The first type of features one can add are the ones related to the author. Accessing at its wikipedia page gives us a lot of different information: looking carrefully at wikidata item field let us select some useful features listed below:
    - `occupation` tells you the author domain.
    - `member of political party` tells you the party at which the author belongs to.
    - `educated at` tells you where the author studied.
    - `country of citizenship` tells you the nationality of the author.
    
    These fields may not exist for all authors (as not all the authors are politicians), but we can actually assign a NaN value when the field does not appear for one author.

2. **Add computed features** : The second type of features we can add are the ones that are directly derived from the initial ones. We selected a bunch of them that will be useful for further analysis:
    - TO BE OPTIONALLY COMPLETED
3. **Add features issued from a sentiment analysis** : The last feature we would like to add is the sentiment carried on by the quotation. Initially we were thinking about a binary sentiment classification: 0 if the sentiment is negative, 1 if it is positive. We could further expand that by classifying the quotations into several categories such as *anger*, *sadness*, *factual* and so on...    
Performing such a text classification task can actually be done using pretrained Deep Neural Networks. XLNet network ([GitHub page](https://github.com/zihangdai/xlnet/) & [Library containing XLNet](https://huggingface.co/transformers/model_doc/xlnet.html)) is close to the state of the art algorithm for classification. Therefore we plan to use it to determine the sentiment contained in each quotation

In [ ]:
# DO THE DATA AUGMENTATION HERE

<a id='clustering'></a>

## Quotations and speakers clustering

The last preprocessing step consist in clustering the quotations as well as the speakers, this clustering will then be used to create a Recommandation Tool. The idea would be to first cluster the quotations and then the speakers such that two quotations/speakers that are in the same cluster are quotations/speakers that carries on similar things/ideas. Performing such a task can be done following this pipeline:
1. The first step is to convert sentences into vectors to be able to further perform the clustering. This task can be achieved using the [SentenceTransformer](https://www.sbert.net/docs/usage/semantic_textual_similarity.html) deep neural network. The vector obtained from this operation cab be then concatenated with the other existing features (that would be converted to one hot vectors if necessary).
2. \[OPTIONAL STEP\] The second step consists in reducing the dimension of the datas before applying the clustering algorithm. This task can be achieved using the [T-stochastic neighbors embeddings](#https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) algorithm or the [Locally Linear Embeddings](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.LocallyLinearEmbedding.html#sklearn.manifold.LocallyLinearEmbedding) algorithm. These two techniques (specially the first one) are efficient non-linear dimensionality reduction methods.
3. The third step is specific to speaker clustering. Indeed the vectorization of quotes as well as the reduction of dimensionality is only applied to quotes. Thus we need to perform an **aggregation** to be able to attribute a vector to each speaker. For each speaker, this aggregation can simply be done by taking the mean of the vectors associated with each of their quotations. 
4. The last step consist in performing the clustering operation. This task can be achieved using [Gaussian Mixture Model](https://scikit-learn.org/stable/modules/mixture.html#mixture) algorithm or  [Spectral Clustering](#https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html#sklearn.cluster.SpectralClustering) method.

However a question arises : which amount of datas should we consider to perform such a clustering ? Cluster using the whole data set seems infeasible.

In [ ]:
# PERFORM CLUSTERING HERE

# upload those modules on jupyter
from sentence_transformers import SentenceTransformer
from sklearn.cluster import SpectralClustering
import pytorch as torch
from tsne_torch import TorchTSNE as TSNE

# Encode data

encoder = SentenceTransformer('all-MiniLM-L6-v2')

# Encode all data into a pytorch tensor NxD
# N = number of sentences (samples)
# D = dimension of a sentence vector
data_tensor = None

# suppose quotes is a dictionary with speakers as keys and their respective quotes as values 

for speaker, sentences in quotes.items():
    # Step 1: encode sentences into a pytorch tensor NxD
    # N = number of sentences (samples)
    # D = dimension of a sentence vector
    quotes_tensor = encoder.encode(sentences, convert_to_tensor=True)
    
    # concatenate tensors by rows
    if data_tensor is None:
        data_tensor = quotes_tensor
    else:
        torch.cat((data_tensor, quotes_tensor), 0)
    
    
# Step 2: T-stochastic neighboor embedding
final_dim = 20 # dim = NxD
data_tensor_emb = TSNE(n_components=final_dim, perplexity=30, n_iter=1000).fit_transform(speaker_tensor) # dim = Nxfinal_dim

# Step 3: contract tensor by mean along rows
speaker_tensor = None

with 0 as i:
    for speaker, sentences in quotes.items():
        speaker_vector = torch.mean(data_tensor_emb[i : len(sentences)], 1)
        i += len(sentences)
        
        if speaker_tensor is None:
            speaker_tensor = speaker_vector
        else:
            torch.cat((speaker_tensor, speaker_vector), 0)



# Step 4: Clustering and actual training

# TODO: vector length normalization?

cluster_model = SpectralClustering()
cluster_model.fit(speaker_tensor_emb)

# TODO: visualization

<a id='Results'></a>

# Generate the results for the final story

<a id='Brexit'></a>

## Analysis of the way Brexit is perceived

Recall that the goal is to analyze the way Brexit is perceived in each Europe country and in each branch (economy) based on the sentiment carried by the quotation. Besides we would like to add the time dimension to this analysis, meaning that we would like to follow the evolution of the overall feelings towards Brexit. A view of the expected result is given below:

Sector Analysis | Country Analysis
- | -
![alt text](Images/brexit_bubbles.png "Sector analysis") | ![alt text](Images/brexit_expected_outcomes.png "Country analysis")

### *Pipeline*


In [ ]:
# BREXIT ANALYSIS